# Setup

Import a few common modules

In [3]:
#CODE for points = 1
# import sklearn, numpy, os

# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)
import numpy as np
import sklearn as sk
assert sk.__version__ >= "0.20"
import os

# Scikit-Learn ≥0.20 is required

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard



# to make this notebook's output stable across runs
np.random.seed(42)


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


# Vanishing/Exploding Gradients Problem

In [4]:
def logit(z):
    return 1 / (1 + np.exp(-z))

In [5]:
z = np.linspace(-5, 5, 200)

## Xavier and He Initialization

In [6]:
#CODE for points = 1
# set activation to relu and kernel initializer to he_normal

#keras.layers.Dense(10, activation=, kernel_initializer=)
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

### Leaky ReLU

In [7]:
#CODE for points = 1

# remember α is the hyperparameter that defines how much the function “leaks”
# set the value of alpha, use the value typically set
def leaky_relu(z, alpha=0.01):
    return np.maximum(alpha*z, z)

Let's train a neural network on Fashion MNIST using the Leaky ReLU:

In [13]:
#CODE for points = 1
#load MNIST dataset from keras
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

4431872/4422102 [==============================] - 0s 0us/step


In [35]:
#CODE for points = 1

# initialize kernel_initializer to "he_normal" and activation function to softmax
tf.random.set_seed(42)
np.random.seed(42)

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
    ])
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

In [17]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [18]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 10s 5ms/step - loss: 1.2819 - accuracy: 0.6229 - val_loss: 0.8886 - val_accuracy: 0.7160
Epoch 2/5
1719/1719 [==============================] - 8s 5ms/step - loss: 0.7955 - accuracy: 0.7361 - val_loss: 0.7130 - val_accuracy: 0.7656
Epoch 3/5
1719/1719 [==============================] - 8s 5ms/step - loss: 0.6816 - accuracy: 0.7721 - val_loss: 0.6427 - val_accuracy: 0.7900
Epoch 4/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.6217 - accuracy: 0.7944 - val_loss: 0.5900 - val_accuracy: 0.8064
Epoch 5/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.5832 - accuracy: 0.8074 - val_loss: 0.5582 - val_accuracy: 0.8200


Now look at what happens if we try to use the ReLU activation function instead:

In [31]:
np.random.seed(42)
tf.random.set_seed(42)

In [32]:
model = keras.models.Sequential()
model.add(keras.layers.Flatten(input_shape=[28, 28]))

model.add(keras.layers.Dense(300, activation="relu", kernel_initializer="he_normal"))

for layer in range(99):
    model.add(keras.layers.Dense(100, activation="relu", kernel_initializer="he_normal"))

model.add(keras.layers.Dense(10, activation="softmax"))

In [37]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [38]:
history = model.fit(X_train_scaled, y_train, epochs=5,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/5
1719/1719 [==============================] - 9s 5ms/step - loss: 0.8591 - accuracy: 0.7092 - val_loss: 0.6172 - val_accuracy: 0.7882
Epoch 2/5
1719/1719 [==============================] - 8s 4ms/step - loss: 0.5678 - accuracy: 0.8034 - val_loss: 0.5268 - val_accuracy: 0.8186
Epoch 3/5
1719/1719 [==============================] - 8s 5ms/step - loss: 0.5051 - accuracy: 0.8241 - val_loss: 0.4882 - val_accuracy: 0.8346
Epoch 4/5
1719/1719 [==============================] - 8s 5ms/step - loss: 0.4712 - accuracy: 0.8357 - val_loss: 0.4636 - val_accuracy: 0.8382
Epoch 5/5
1719/1719 [==============================] - 7s 4ms/step - loss: 0.4486 - accuracy: 0.8433 - val_loss: 0.4468 - val_accuracy: 0.8466


Not great at all, we suffered from the vanishing/exploding gradients problem.

# Batch Normalization

In [20]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [21]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_4 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_5 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [22]:
bn1 = model.layers[1]
[(var.name, var.trainable) for var in bn1.variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [ ]:
#bn1.updates #deprecated

In [24]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [25]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 14s 8ms/step - loss: 0.8293 - accuracy: 0.7221 - val_loss: 0.5539 - val_accuracy: 0.8160
Epoch 2/5
1719/1719 [==============================] - 13s 7ms/step - loss: 0.5703 - accuracy: 0.8035 - val_loss: 0.4792 - val_accuracy: 0.8378
Epoch 3/5
1719/1719 [==============================] - 13s 8ms/step - loss: 0.5161 - accuracy: 0.8212 - val_loss: 0.4424 - val_accuracy: 0.8490
Epoch 4/5
1719/1719 [==============================] - 13s 7ms/step - loss: 0.4789 - accuracy: 0.8315 - val_loss: 0.4212 - val_accuracy: 0.8566
Epoch 5/5
1719/1719 [==============================] - 13s 7ms/step - loss: 0.4548 - accuracy: 0.8404 - val_loss: 0.4050 - val_accuracy: 0.8610


Sometimes applying BN before the activation function works better (there's a debate on this topic). Moreover, the layer before a `BatchNormalization` layer does not need to have bias terms, since the `BatchNormalization` layer some as well, it would be a waste of parameters, so you can set `use_bias=False` when creating those layers:

In [39]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [ ]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [ ]:
history = model.fit(X_train, y_train, epochs=5,
                    validation_data=(X_valid, y_valid))

Epoch 1/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4748 - accuracy: 0.8401 - val_loss: 0.4176 - val_accuracy: 0.8572
Epoch 2/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4601 - accuracy: 0.8412 - val_loss: 0.4052 - val_accuracy: 0.8594
Epoch 3/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4494 - accuracy: 0.8454 - val_loss: 0.3969 - val_accuracy: 0.8628
Epoch 4/5
1719/1719 [==============================] - 10s 6ms/step - loss: 0.4351 - accuracy: 0.8503 - val_loss: 0.3896 - val_accuracy: 0.8652
Epoch 5/5
1719/1719 [==============================] - 11s 6ms/step - loss: 0.4236 - accuracy: 0.8524 - val_loss: 0.3832 - val_accuracy: 0.8688


# Faster Optimizers

## Momentum optimization

In [26]:
#CODE for points = 1
# initialize lr and momentum to typical values

optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

## Nesterov Accelerated Gradient

In [27]:
#CODE for points = 1
# initialize lr and momentum to typical values. Set nesterov so that it is used

optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

# Avoiding Overfitting Through Regularization

## $\ell_1$ and $\ell_2$ regularization

In [28]:
#CODE for points = 0.5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

In [40]:
#CODE for points = 1

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/
# nadam optimizer

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))


Epoch 1/2
1719/1719 [==============================] - 18s 10ms/step - loss: 1.5816 - accuracy: 0.8115 - val_loss: 0.7197 - val_accuracy: 0.8302
Epoch 2/2
1719/1719 [==============================] - 17s 10ms/step - loss: 0.7210 - accuracy: 0.8276 - val_loss: 0.6862 - val_accuracy: 0.8360


In [42]:
#CODE for points = .5

# Use syntax for assigning l2 regularization with a factor 0.01 as given here - https://keras.io/api/layers/regularizers/

from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))


In [43]:
#CODE for points = 1

# activation function initialized as softmax
# nadam optimizer


model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Epoch 1/2
1719/1719 [==============================] - 21s 11ms/step - loss: 1.6630 - accuracy: 0.8122 - val_loss: 0.7201 - val_accuracy: 0.8308
Epoch 2/2
1719/1719 [==============================] - 19s 11ms/step - loss: 0.7188 - accuracy: 0.8266 - val_loss: 0.6806 - val_accuracy: 0.8402
